# Class 4:  Webscraping
# COMPLETE

Webscraping is a technique to extract data from a website.  Today we'll extract player passing data for the 2018 regular season from nfl.com.

Start by getting familar with this website (recommend using Google Chrome):  
http://www.nfl.com/stats/categorystats?tabSeq=0&season=2018&seasonType=REG&statisticCategory=PASSING&d-447263-p=1

We will be interested in extracting the player's name, team url, position, completions, attempts, yards, and touchdowns.

Right-click on 	Ben Roethlisberger's name and select "Inspect".  This opens the web developer tool, which shows the html code used for this webpage.  We will pick through the html tags in order to extract the information we want.  What html tag contains the information we want?

## Step 1: Make a BeautifulSoup Object

First, we need to establish the connection to the website, and create a beatiful soup object with all of the data (HTML) within the website.  To do this, we'll need to import both the 'beautifulSoup' library (this is the main library we'll be using to parse the website), and the 'requests' library (this will help us establish our connection to the website).

To create a beautiful soup object (we'll name our variable 'soup'), you need to pass it the content of a website, and you can specify which type of parser this oject should use to parse the website.  We specify this parser as the second argument when creating a BeautifulSoup object.  Our options:  'html.parser', 'lxml', 'xml', 'html5lib'.  We would recommend using 'html.parser' to start with as this is normally sufficient.  If you know with certainty that it's not working as expected, try a different parser.

The beautifulSoup documentation page is incredibly helpful with gaining some basic understanding of the beautifulSoup library: https://www.crummy.com/software/BeautifulSoup/bs4/doc/#contents-and-childrenIn general, the 'html.parser'

In [1]:
# import the libraries we need
import requests
from bs4 import BeautifulSoup

# have url hold the website's url
url = "http://www.nfl.com/stats/categorystats?tabSeq=0&season=2018&seasonType=REG&statisticCategory=PASSING&d-447263-p=1"

In [2]:
# Query the website and return the html to the variable 'website'
website = requests.get(url)
website

<Response [200]>

In [3]:
# Parse the html in the 'website' variable, and store it in a 
# beautiful soup form
# NOTE: there are other parsers availabe (we are using html.parser)
#       sometimes the xlml parser may be what you want
soup = BeautifulSoup(website.content, 'html.parser')

In [4]:
# Look at it...looks like the same stuff we get when we open the developer tool on the website
soup

## Step 2:  Extract information from table

Now that we have a beatifulSoup object, we can use all of the class methods available to search and extract the desired data from this object.  Let's think through how we will store the information we extract from this website.  Recall, we want to collect:  name, teamURL, completions, attempts, yards, and touchdowns for each player that is listed under the passing stats the 2018 regular season.  

Ultimately, we will want to put this table in a pandas dataframe.  Recall that we can make a pandas dataframe from a dictionary of lists -- we can also make a pandas dictionary with a list of dictionaries.  We will opt for the later, that is, each element in the list is a dictionary, where each dictionary holds the information for one player.  The dictionary keys will be the same for each player:  'name','teamURL','position' 'completions', 'attempts', 'yards', and 'TD'.  Our data structure will look like this:

passing_2018stats = 
[
    {'name': player1 name, 'teamURL': url of team, 'position': position of player1, 'completions': completions of player1, 'attempts': attempts of player1, 'yards': passing yards of player1, 'TD': TDs of player1}, 
    {'name': player2 name, 'teamURL': url of team, 'position': position of player2, 'completions': completions of player2, 'attempts': attempts of player2, 'yards': passing yards of player2, 'TD': TDs of player1},
    etc...
]

When extracting information from a website, we generally retrieve the information as strings.  We might ultimately want to change these data types to something else (an int, or float), but for this example, we'll just focus on getting the desired information, knowing that we could convert it to a different data type if we wanted.  

Inspecting the website (from your chrome browser), we can see that each player is organized under a 'tr' tag with the attribute 'class' = 'odd' or 'class' = 'even'. 

Using the 'find_all' method of our beatifulSoup object, we can identify all of the 'tr' tags.  (Note: the 'find' method will only return the first instance of the 'tr' tag.

In [7]:
# returns the first instance of the 'tr' tag
soup.find('tr')

In [8]:
# returns a list of all tr tags
soup.find_all('tr')

Recall that there is a class associated with every player ('odd' or 'even'), we can specify our in our 'find_all' method even further using the class attribute.

Play around printing various things to determine if you are able to capture what you expect.

In [9]:
# returns a list of all tr tags
soup.find_all('tr', class_ = 'odd')

Now that we know what tag we want and how to get it, let's save it to 'player_list'

In [10]:
# Let's find all the event-group classes on the webpage
player_list = soup.find_all('tr')
# Let's look to see how many months are on the site
print(len(player_list))

51


In [11]:
# But we only expected 50...so let's try and figure out what tr tag we didn't want...
player_list[0]

# So we'll need to skip over this one!
player_list = player_list[1:]
len(player_list)

50

In [12]:
print(type(player_list[0]))

<class 'bs4.element.Tag'>


## Step 2(a)
A good inital step is to see extract the information for a single player.  If we can do it for one, we should easily be able to do it for the rest.  Let's start by getting Ben Roethlisberger's data.

In [14]:
# Let's look at Ben Roethlisberger
player = player_list[0]
player

Looks like we can simply index into the 'player_list' to extract a single row.  Now let's figure out how to get the columns -- what is the html tag we want?

In [15]:
# Save all the columns for the first player
playerCols = player.find_all('td')
playerCols

In [16]:
# Quick check -- how many columns should we have?
len(playerCols)

20

In [17]:
# Good to know... you can specify by class too
player.find_all('td', class_ = "right")

Extract and print out Ben Roethlisberger's name, team, team url, and yards

In [18]:
# Name
playerCols[1].find('a').get_text()

'Ben Roethlisberger'

In [19]:
# team url
playerCols[2].find('a')['href']

'/teams/pittsburghsteelers/profile?team=PIT'

In [20]:
# position
playerCols[3].get_text()

'QB'

In [21]:
# completions
playerCols[4].get_text().strip()

'452'

In [22]:
# attempts
playerCols[5].get_text().strip()

'675'

In [23]:
# yards
playerCols[8].get_text().strip()

'5,129'

In [24]:
# touchdowns
playerCols[11].get_text().strip()

'34'

## Step 2(b) 
Now that we were able to find Ben Roethlisberger's data, let's loop through all rows, saving each player's information to a dictionary, and then appending that dictionary to the list 'passing_2018stats'.

In [25]:
passing_2018stats = []

for player in player_list:
    playerCols = player.find_all('td')
    colDict = {}
    # Name
    colDict['name'] = playerCols[1].find('a').get_text()
    # Team URL
    colDict['teamURL'] = playerCols[2].find('a')['href']
    # Position
    colDict['position'] = playerCols[3].get_text()
    # Completions
    colDict['completions'] = playerCols[4].get_text().strip()
    # Attempts
    colDict['attempts'] = playerCols[5].get_text().strip()
    # Yards
    colDict['yards'] = playerCols[8].get_text().strip()
    # Touchdowns
    colDict['TDs'] = playerCols[11].get_text().strip()
    passing_2018stats.append(colDict)

In [26]:
passing_2018stats

# Step 3: Turn the list of dictionaries to a pandas dataframe

In [27]:
import pandas as pd

In [28]:
playerdf = pd.DataFrame(passing_2018stats)
playerdf.head()

,TDs,attempts,completions,name,position,teamURL,yards
0,34,675,452,Ben Roethlisberger,QB,/teams/pittsburghsteelers/profile?team=PIT,"5,129"
1,50,580,383,Patrick Mahomes,QB,/teams/kansascitychiefs/profile?team=KC,"5,097"
2,35,608,422,Matt Ryan,QB,/teams/atlantafalcons/profile?team=ATL,"4,924"
3,32,561,364,Jared Goff,QB,/teams/losangelesrams/profile?team=LA,"4,688"
4,39,639,430,Andrew Luck,QB,/teams/indianapoliscolts/profile?team=IND,"4,593"
